In [2]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [3]:
from collections import defaultdict
from gensim import corpora

documents = [
    "Human machine interface for lab abc computer applications",
    "A survey of user opinion of computer system response time",
    "The EPS user interface management system",
    "System and human system engineering testing of EPS",
    "Relation of user perceived response time to error measurement",
    "The generation of random binary unordered trees",
    "The intersection graph of paths in trees",
    "Graph minors IV Widths of trees and well quasi ordering",
    "Graph minors A survey",
]

# remove common words and tokenize
stoplist = set('for a of the and to in'.split())
texts = [
    [word for word in document.lower().split() if word not in stoplist]
    for document in documents
]

# remove words that appear only once
frequency = defaultdict(int)
for text in texts:
    for token in text:
        frequency[token] += 1

texts = [
    [token for token in text if frequency[token] > 1]
    for text in texts
]

dictionary = corpora.Dictionary(texts)
corpus = [dictionary.doc2bow(text) for text in texts]

2023-03-29 19:13:13,610 : INFO : adding document #0 to Dictionary<0 unique tokens: []>
2023-03-29 19:13:13,611 : INFO : built Dictionary<12 unique tokens: ['computer', 'human', 'interface', 'response', 'survey']...> from 9 documents (total 29 corpus positions)
2023-03-29 19:13:13,612 : INFO : Dictionary lifecycle event {'msg': "built Dictionary<12 unique tokens: ['computer', 'human', 'interface', 'response', 'survey']...> from 9 documents (total 29 corpus positions)", 'datetime': '2023-03-29T19:13:13.612840', 'gensim': '4.3.1', 'python': '3.10.4 | packaged by conda-forge | (main, Mar 30 2022, 08:38:02) [MSC v.1916 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.19044-SP0', 'event': 'created'}


In [4]:
from gensim import models

tfidf = models.TfidfModel(corpus)  # step 1 -- initialize a model

2023-03-29 19:13:22,695 : INFO : collecting document frequencies
2023-03-29 19:13:22,696 : INFO : PROGRESS: processing document #0
2023-03-29 19:13:22,696 : INFO : TfidfModel lifecycle event {'msg': 'calculated IDF weights for 9 documents and 12 features (28 matrix non-zeros)', 'datetime': '2023-03-29T19:13:22.696798', 'gensim': '4.3.1', 'python': '3.10.4 | packaged by conda-forge | (main, Mar 30 2022, 08:38:02) [MSC v.1916 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.19044-SP0', 'event': 'initialize'}


In [5]:
doc_bow = [(0, 1), (1, 1)]
print(tfidf[doc_bow])  # step 2 -- use the model to transform vectors


[(0, 0.7071067811865476), (1, 0.7071067811865476)]


In [6]:
corpus_tfidf = tfidf[corpus]
for doc in corpus_tfidf:
    print(doc)

[(0, 0.5773502691896257), (1, 0.5773502691896257), (2, 0.5773502691896257)]
[(0, 0.44424552527467476), (3, 0.44424552527467476), (4, 0.44424552527467476), (5, 0.3244870206138555), (6, 0.44424552527467476), (7, 0.3244870206138555)]
[(2, 0.5710059809418182), (5, 0.4170757362022777), (7, 0.4170757362022777), (8, 0.5710059809418182)]
[(1, 0.49182558987264147), (5, 0.7184811607083769), (8, 0.49182558987264147)]
[(3, 0.6282580468670046), (6, 0.6282580468670046), (7, 0.45889394536615247)]
[(9, 1.0)]
[(9, 0.7071067811865475), (10, 0.7071067811865475)]
[(9, 0.5080429008916749), (10, 0.5080429008916749), (11, 0.695546419520037)]
[(4, 0.6282580468670046), (10, 0.45889394536615247), (11, 0.6282580468670046)]


In [7]:
lsi_model = models.LsiModel(corpus_tfidf, id2word=dictionary, num_topics=2)  # initialize an LSI transformation
corpus_lsi = lsi_model[corpus_tfidf]  # create a double wrapper over the original corpus: bow->tfidf->fold-in-lsi

2023-03-29 19:15:56,178 : INFO : using serial LSI version on this node
2023-03-29 19:15:56,179 : INFO : updating model with new documents
2023-03-29 19:15:56,179 : INFO : preparing a new chunk of documents
2023-03-29 19:15:56,180 : INFO : using 100 extra samples and 2 power iterations
2023-03-29 19:15:56,180 : INFO : 1st phase: constructing (12, 102) action matrix
2023-03-29 19:15:56,187 : INFO : orthonormalizing (12, 102) action matrix
2023-03-29 19:15:56,191 : INFO : 2nd phase: running dense svd on (12, 9) matrix
2023-03-29 19:15:56,194 : INFO : computing the final decomposition
2023-03-29 19:15:56,195 : INFO : keeping 2 factors (discarding 47.565% of energy spectrum)
2023-03-29 19:15:56,196 : INFO : processed documents up to #9
2023-03-29 19:15:56,197 : INFO : topic #0(1.594): -0.703*"trees" + -0.538*"graph" + -0.402*"minors" + -0.187*"survey" + -0.061*"system" + -0.060*"time" + -0.060*"response" + -0.058*"user" + -0.049*"computer" + -0.035*"interface"
2023-03-29 19:15:56,198 : INFO

In [8]:
lsi_model.print_topics(2)

2023-03-29 19:16:31,037 : INFO : topic #0(1.594): -0.703*"trees" + -0.538*"graph" + -0.402*"minors" + -0.187*"survey" + -0.061*"system" + -0.060*"time" + -0.060*"response" + -0.058*"user" + -0.049*"computer" + -0.035*"interface"
2023-03-29 19:16:31,038 : INFO : topic #1(1.476): -0.460*"system" + -0.373*"user" + -0.332*"eps" + -0.328*"interface" + -0.320*"time" + -0.320*"response" + -0.293*"computer" + -0.280*"human" + -0.171*"survey" + 0.161*"trees"


[(0,
  '-0.703*"trees" + -0.538*"graph" + -0.402*"minors" + -0.187*"survey" + -0.061*"system" + -0.060*"time" + -0.060*"response" + -0.058*"user" + -0.049*"computer" + -0.035*"interface"'),
 (1,
  '-0.460*"system" + -0.373*"user" + -0.332*"eps" + -0.328*"interface" + -0.320*"time" + -0.320*"response" + -0.293*"computer" + -0.280*"human" + -0.171*"survey" + 0.161*"trees"')]

In [9]:
# both bow->tfidf and tfidf->lsi transformations are actually executed here, on the fly
for doc, as_text in zip(corpus_lsi, documents):
    print(doc, as_text)

[(0, -0.06600783396090226), (1, -0.5200703306361855)] Human machine interface for lab abc computer applications
[(0, -0.19667592859142308), (1, -0.760956316770005)] A survey of user opinion of computer system response time
[(0, -0.08992639972446229), (1, -0.7241860626752511)] The EPS user interface management system
[(0, -0.07585847652177985), (1, -0.6320551586003432)] System and human system engineering testing of EPS
[(0, -0.1015029918498001), (1, -0.5737308483002955)] Relation of user perceived response time to error measurement
[(0, -0.7032108939378318), (1, 0.16115180214025618)] The generation of random binary unordered trees
[(0, -0.8774787673119837), (1, 0.16758906864659182)] The intersection graph of paths in trees
[(0, -0.9098624686818582), (1, 0.14086553628718773)] Graph minors IV Widths of trees and well quasi ordering
[(0, -0.6165825350569277), (1, -0.05392907566389537)] Graph minors A survey


In [10]:
import os
import tempfile

with tempfile.NamedTemporaryFile(prefix='model-', suffix='.lsi', delete=False) as tmp:
    lsi_model.save(tmp.name)  # same for tfidf, lda, ...

loaded_lsi_model = models.LsiModel.load(tmp.name)

os.unlink(tmp.name)

2023-03-29 19:18:43,813 : INFO : Projection lifecycle event {'fname_or_handle': 'C:\\Users\\biehl\\AppData\\Local\\Temp\\model-tma3192s.lsi.projection', 'separately': 'None', 'sep_limit': 10485760, 'ignore': frozenset(), 'datetime': '2023-03-29T19:18:43.813765', 'gensim': '4.3.1', 'python': '3.10.4 | packaged by conda-forge | (main, Mar 30 2022, 08:38:02) [MSC v.1916 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.19044-SP0', 'event': 'saving'}
2023-03-29 19:18:43,815 : INFO : saved C:\Users\biehl\AppData\Local\Temp\model-tma3192s.lsi.projection
2023-03-29 19:18:43,815 : INFO : LsiModel lifecycle event {'fname_or_handle': 'C:\\Users\\biehl\\AppData\\Local\\Temp\\model-tma3192s.lsi', 'separately': 'None', 'sep_limit': 10485760, 'ignore': ['projection', 'dispatcher'], 'datetime': '2023-03-29T19:18:43.815025', 'gensim': '4.3.1', 'python': '3.10.4 | packaged by conda-forge | (main, Mar 30 2022, 08:38:02) [MSC v.1916 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.19044-SP0', 'event': 'saving

In [1]:
!jupyter nbconvert --to html "Gensim Code Exploration 3.ipynb"

[NbConvertApp] Converting notebook Gensim Code Exploration 3.ipynb to html
[NbConvertApp] Writing 606182 bytes to Gensim Code Exploration 3.html
